## 목차 (Table of Contents)

**준비 과정**
- [라이브러리 설치 및 불러오기](#준비하기-라이브러리-설치-및-불러오기)
- [MNIST 데이터셋 불러오기](#MNIST-데이터셋-불러오기)

**실습 (Practice)**
1. [행렬 곱셈과 활용](#1-행렬-곱셈과-활용)
2. [역행렬과 행렬식](#2-역행렬과-행렬식)
3. [K-means의 한계와 Feature Transformation](#3-K-means의-한계와-Feature-Transformation)
4. [최소제곱 데이터 피팅](#4-최소제곱-데이터-피팅)

## 준비하기: 라이브러리 설치 및 불러오기

In [ ]:
# 라이브러리 설치
import subprocess
import sys

def install_if_not_exists(package):
    try:
        # sklearn은 scikit-learn으로 import 해야 합니다.
        import_name = 'sklearn' if package == 'scikit-learn' else package
        __import__(import_name)
    except ImportError:
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])

# 모든 실습에 필요한 라이브러리 목록
required_packages = ["numpy", "matplotlib", "scikit-learn"]
for package in required_packages:
    install_if_not_exists(package)


# 라이브러리 불러오기
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.datasets import make_blobs, make_circles, fetch_openml

# 전체 실습의 재현성을 위해 랜덤 시드를 고정합니다.
np.random.seed(0)

## MNIST 데이터셋 불러오기
scikit-learn의 `fetch_openml`을 사용하여 MNIST 손글씨 숫자 데이터셋을 불러옵니다.
데이터는 784개의 픽셀(28x28)로 구성된 이미지이며, 0~255 값을 갖습니다.
K-means와 PCA 실습에서 사용할 수 있도록 255로 나누어 정규화합니다.

In [ ]:
print("MNIST 데이터셋을 불러오는 중... (몇 분 정도 소요될 수 있습니다)")
try:
    # as_frame=False : numpy array로 받기
    # parser='auto' : 최신 scikit-learn에서 권장하는 파서
    mnist = fetch_openml('mnist_784', version=1, as_frame=False, parser='auto')
    
    X_mnist_data = mnist.data / 255.0  # 정규화
    y_mnist_data = mnist.target.astype(int)
    
    print("MNIST 데이터셋 로드 완료.")
    print(f"데이터 형태: {X_mnist_data.shape}")
    print(f"레이블 형태: {y_mnist_data.shape}")
except Exception as e:
    print(f"데이터셋 로드 중 오류 발생: {e}")
    print("인터넷 연결을 확인하거나, 잠시 후 다시 시도해주세요.")
    X_mnist_data, y_mnist_data = None, None

## 1. 행렬 곱셈과 활용

### 1.1. 행렬 곱셈 (Matrix-Matrix Multiplication)
두 행렬의 곱셈 $C = AB$는 첫 번째 행렬 $A$의 열 개수와 두 번째 행렬 $B$의 행 개수가 같아야 가능합니다.
(m, p) 크기 행렬과 (p, n) 크기 행렬을 곱하면 결과는 (m, n) 크기의 행렬이 됩니다.
행렬 곱은 **선형 변환의 연속(합성)**을 의미하며, 교환 법칙($AB \neq BA$)이 성립하지 않는다는 특징이 있습니다.

In [ ]:
# 실습: 두 행렬의 값을 바꿔보거나, 행렬의 크기를 바꿔서 곱셈을 시도해보세요.
# (m, p) x (p, n) 크기의 행렬 곱셈
A_mul = np.array([
    [1, 2, 3],
    [4, 5, 6]
])
B_mul = np.array([
    [10, 11],
    [20, 21],
    [30, 31]
])

# 행렬 곱셈. A_mul의 열 수(3)와 B_mul의 행 수(3)가 같아야 합니다.
C_mul = A_mul @ B_mul  # 결과는 (2, 2) 행렬

print("행렬 A (2x3):\n", A_mul)
print("행렬 B (3x2):\n", B_mul)
print("-" * 20)
print("행렬 곱 AB:\n", C_mul)
print(f"AB의 크기: {C_mul.shape}")

### 1.2. 벡터의 외적 (Outer Product)
두 벡터 $a$(크기 m)와 $b$(크기 n)의 외적은 $m \times n$ 크기의 행렬을 생성합니다.
$a$의 각 요소와 $b$의 각 요소의 모든 조합을 곱한 행렬로, `np.outer(a, b)`로 계산합니다.
외적은 두 벡터의 상호작용을 행렬로 표현하여, 데이터 분석이나 머신러닝 모델의 가중치 표현 등에서 활용됩니다.

In [ ]:
# 실습: a_vec, b_vec의 값이나 크기를 바꿔보며 외적의 결과가 어떻게 변하는지 확인해보세요.
a_vec = np.array([1, 2, 3])
b_vec = np.array([10, 20])

# np.outer() 함수를 사용하여 두 벡터의 외적을 계산합니다.
outer_product = np.outer(a_vec, b_vec)

print("벡터 a:", a_vec)
print("벡터 b:", b_vec)
print("-" * 20)
print("a와 b의 외적 (np.outer(a,b)):\n", outer_product)
print(f"외적 행렬의 크기: {outer_product.shape}")

### 1.3. 활용 1: 기하 변환 (Geometric Transformations)
행렬 곱셈은 벡터를 특정 방식으로 변환하는 강력한 도구입니다. 예를 들어, 2D 벡터를 $\\theta$만큼 **회전**시키는 변환은 아래와 같은 회전 행렬 $R$을 곱하여 수행할 수 있습니다.

```
      [ cos(θ)  -sin(θ) ]
R =   [ sin(θ)   cos(θ) ]
```

`변환된 벡터 = R @ 원본 벡터`

In [ ]:
# 실습: 벡터 v의 값이나, 회전 각도 theta를 바꿔서 변환 결과를 확인해보세요.
# 변환할 2D 벡터를 정의합니다.
v = np.array([1, 0]) # x축 방향의 단위 벡터

# 45도 회전을 위한 변환 행렬을 생성합니다.
theta = np.radians(45)
R_mat = np.array([[np.cos(theta), -np.sin(theta)],
              [np.sin(theta),  np.cos(theta)]])

# 회전 변환을 적용합니다.
v_rotated = R_mat @ v

print("원본 벡터 v:", v)
print("45도 회전 행렬 R:\n", np.round(R_mat, 2))
print("회전된 벡터 v_rotated:", np.round(v_rotated, 2))

In [ ]:
# 시각화
origin = np.array([0, 0])

all_points_x_rot = [0, v[0], v_rotated[0]]
all_points_y_rot = [0, v[1], v_rotated[1]]

plt.figure(figsize=(6,6))
plt.quiver(*origin, *v, angles='xy', scale_units='xy', scale=1, color='r', label=f'Original v = {v}')
plt.quiver(*origin, *v_rotated, angles='xy', scale_units='xy', scale=1, color='g', label=f'Rotated v = ({v_rotated[0]:.2f}, {v_rotated[1]:.2f})')

# 동적으로 계산된 범위에 여백을 주어 x, y축 범위를 설정합니다.
plt.xlim(min(all_points_x_rot) - 1, max(all_points_x_rot) + 1)
plt.ylim(min(all_points_y_rot) - 1, max(all_points_y_rot) + 1)

plt.title('Geometric Transformation (Rotation)')
plt.xlabel('x-axis')
plt.ylabel('y-axis')
plt.grid()
plt.legend()
plt.gca().set_aspect('equal', adjustable='box')
plt.show()

### 1.4. 활용 2: 선형 변환의 합성 (Composition)
두 가지 선형 변환을 연속으로 적용하는 것은, 각 변환에 해당하는 두 행렬을 먼저 곱하여 얻은 **합성 행렬**을 한 번 적용하는 것과 같습니다.
예를 들어, 30도 회전 후 60도 회전을 적용하는 것은, 90도 회전 행렬을 한 번 적용하는 것과 동일합니다.

$(R_{60} @ R_{30}) @ v = R_{60} @ (R_{30} @ v)$

In [ ]:
# 실습: 벡터 v_comp의 값이나, 두 회전 각도를 바꿔보며 결과를 확인해보세요.
# 원본 벡터를 정의합니다.
v_comp = np.array([2, 1])

# 30도 회전 변환
theta_30 = np.radians(30)
R_30 = np.array([[np.cos(theta_30), -np.sin(theta_30)],
                 [np.sin(theta_30),  np.cos(theta_30)]])

# 60도 회전 변환
theta_60 = np.radians(60)
R_60 = np.array([[np.cos(theta_60), -np.sin(theta_60)],
                 [np.sin(theta_60),  np.cos(theta_60)]])

# 방법 1: 변환을 순차적으로 적용합니다 (30도 회전 후 60도 회전).
v_rotated_seq = R_60 @ (R_30 @ v_comp)

# 방법 2: 변환 행렬을 먼저 곱하여 합성한 후, 한 번에 적용합니다.
R_90 = R_60 @ R_30
v_rotated_combined = R_90 @ v_comp

print("원본 벡터:", v_comp)
print("-" * 20)
print(f"순차 변환 결과 (30도 -> 60도):\n", np.round(v_rotated_seq, 2))
print(f"합성 변환 결과 (90도):\n", np.round(v_rotated_combined, 2))

## 2. 역행렬과 행렬식

### 2.1. 역행렬 (Matrix Inverse)
어떤 정방 행렬(square matrix) $A$에 대해, 곱했을 때 단위 행렬($I$)이 되는 행렬 $B$가 존재한다면, $B$를 $A$의 **역행렬(inverse matrix)**이라 부르고 $A^{-1}$로 표기합니다.

$A @ A^{-1} = A^{-1} @ A = I$

역행렬은 어떤 변환을 '되돌리는'(undo) 변환에 해당하며, 연립 방정식을 푸는 데 핵심적인 역할을 합니다.
**모든 정방 행렬이 역행렬을 갖는 것은 아닙니다.**

- `np.linalg.inv()` 함수로 역행렬을 계산할 수 있습니다.

In [ ]:
# 실습: 아래 행렬 A_inv_source의 값을 바꿔보며 역행렬을 계산해보세요.
# 역행렬을 계산할 2x2 정방 행렬 정의
A_inv_source = np.array([
    [1, 1],
    [1, 2]
])

# 역행렬 계산
try:
    A_inverse = np.linalg.inv(A_inv_source)
    print("원본 행렬 A:\n", A_inv_source)
    print("-" * 20)
    print("A의 역행렬 A^{-1}:\n", A_inverse)
    print("-" * 20)

    # A @ A^{-1}가 단위 행렬인지 확인 (부동소수점 오차를 고려)
    identity_check = A_inv_source @ A_inverse
    print("A @ A^{-1} 결과:\n", identity_check)

    # np.allclose()를 이용한 단위 행렬 검증
    is_identity = np.allclose(identity_check, np.identity(2))
    print(f"\n결과가 단위 행렬과 매우 가깝습니까? -> {is_identity}")

except np.linalg.LinAlgError as e:
    print("오류:", e)
    print("이 행렬은 역행렬을 가지지 않습니다 (특이 행렬).")

### 2.2. 행렬식 (Determinant)
행렬식은 정방 행렬이 갖는 고유한 스칼라 값으로, `np.linalg.det()`으로 계산합니다.
기하학적으로 행렬이 변환시키는 공간의 '부피'가 얼마나 변하는지를 나타냅니다.

- **$\det(A) \neq 0$**: 행렬 $A$는 역행렬을 가집니다 (가역 행렬, Invertible).
- **$\det(A) = 0$**: 행렬 $A$는 역행렬을 가지지 않습니다 (특이 행렬, Singular).

따라서 행렬식은 역행렬의 존재 여부를 판별하는 중요한 지표입니다.

In [ ]:
# 실습: A_inv_source의 행렬식을 계산해보고, 아래 특이 행렬의 행렬식과 비교해보세요.
# 가역 행렬 (Invertible Matrix)
det_A = np.linalg.det(A_inv_source)
print(f"가역 행렬 A:\n{A_inv_source}")
print(f"A의 행렬식: {det_A:.1f}")
print("-" * 20)

# 특이 행렬 (Singular Matrix)
singular_matrix_det_check = np.array([
    [1, 2],
    [2, 4]
])
det_S = np.linalg.det(singular_matrix_det_check)
print(f"특이 행렬 S:\n{singular_matrix_det_check}")
print(f"S의 행렬식: {det_S:.1f}")

### 2.3. 역행렬 계산 시도와 특이 행렬
역행렬이 존재하지 않는 **특이 행렬(Singular Matrix)**에 `np.linalg.inv()`를 사용하면 `LinAlgError`가 발생합니다.

In [ ]:
# 실습: 특이 행렬에 역행렬 계산을 시도하면 LinAlgError가 발생하는 것을 확인합니다.
singular_matrix = np.array([
    [1, 2],
    [2, 4]
])

print("특이 행렬 B:\n", singular_matrix)
print("-" * 20)

# 역행렬 계산 시도
try:
    B_inverse = np.linalg.inv(singular_matrix)
    print("B의 역행렬:\n", B_inverse)
except np.linalg.LinAlgError as e:
    print("np.linalg.inv(B) 실행 시 오류 발생:")
    print(f"-> {e}")

## 3. K-means의 한계와 Feature Transformation
K-means는 군집이 구형(spherical)이며, 중심점으로부터의 거리로 잘 구분될 것을 가정합니다.
이 가정이 깨지는 데이터에서는 잘 동작하지 않습니다. 이 한계를 확인하고, Feature Transformation을 통해 해결해봅니다.

### 3.1. 실패 사례: 동심원 데이터에 K-means 적용
`make_circles`로 생성된 동심원 데이터에 K-means를 적용하면, 거리 기반 군집화의 한계로 인해 올바르게 분류하지 못합니다.

In [ ]:
# 동심원 데이터 생성
# 실습: n_samples, factor, noise 값을 바꿔보며 데이터 분포가 어떻게 변하는지 확인해보세요.
X_circles, y_circles = make_circles(n_samples=500, factor=0.5, noise=0.05, random_state=0)

# 1. 원본 데이터에 K-means 적용 (실패 사례)
kmeans_fail = KMeans(n_clusters=2, init='k-means++', n_init=1, random_state=0)
y_kmeans_fail = kmeans_fail.fit_predict(X_circles)

plt.figure(figsize=(8, 6))
plt.scatter(X_circles[:, 0], X_circles[:, 1], c=y_kmeans_fail, s=50, cmap='viridis')
plt.title('K-Means Fails on Concentric Circles')
# 가로, 세로 비율을 동일하게 설정하여 원이 제대로 보이게 합니다.
plt.gca().set_aspect('equal', adjustable='box')
plt.grid()
plt.show()

### 3.2. Feature Transformation
데이터의 표현 방식을 바꾸어 K-means가 인식할 수 있는 형태로 만들어줍니다. 기존의 (x, y) 직교 좌표계를 **거리(r)**와 **각도($\\theta$)**를 나타내는 **극 좌표계**로 변환합니다.
변환된 좌표계에서 데이터가 어떻게 보이는지 확인해봅시다.

In [ ]:
# 피처 변환: 직교 좌표계 -> 극 좌표계
r = np.sqrt(X_circles[:, 0] ** 2 + X_circles[:, 1] ** 2)
theta = np.arctan2(X_circles[:, 1], X_circles[:, 0])

# 변환된 데이터를 시각화하여 구조를 확인합니다.
# 이 단계에서는 아직 클러스터링을 적용하기 전이므로 모든 점을 같은 색으로 표시합니다.
plt.figure(figsize=(8, 6))
plt.scatter(theta, r, s=50, alpha=0.7)
plt.title('Transformed Data (Polar Coordinates)')
plt.xlabel('Theta (angle)')
plt.ylabel('R (distance from origin)')
plt.grid()
plt.show()

### 3.3. 변환된 공간에서 K-means 적용 및 시각화
극 좌표계로 변환된 데이터는 거리($r$) 축만으로도 두 군집이 선형적으로 분리됩니다.
이제 이 $r$ 피처에 K-means를 적용하고, 그 결과를 극 좌표계에 시각화하여 확인합니다.

In [ ]:
# 변환된 데이터(거리 r)에 K-means 적용
# r.reshape(-1, 1)는 1차원 배열 r을 2차원 열 벡터로 변환합니다.
kmeans_success = KMeans(n_clusters=2, init='k-means++', n_init=1, random_state=0)
y_kmeans_success = kmeans_success.fit_predict(r.reshape(-1, 1))

# 변환된 공간에서 클러스터링 결과 시각화
plt.figure(figsize=(8, 6))
plt.scatter(theta, r, c=y_kmeans_success, s=50, cmap='viridis', alpha=0.7)
plt.title('K-Means Clustering in Transformed Space')
plt.xlabel('Theta (angle)')
plt.ylabel('R (distance from origin)')
plt.grid()
plt.show()

### 3.4. 원본 좌표계에 결과 시각화
마지막으로, 변환된 공간에서 성공적으로 찾아낸 클러스터 레이블을 원본 (x, y) 좌표계의 데이터에 적용하여 시각화합니다.
Feature Transformation을 통해 K-means가 복잡한 데이터 구조를 학습할 수 있게 되었음을 확인할 수 있습니다.

In [ ]:
# 피처 변환 후 K-means가 성공적으로 분류한 결과를 원본 데이터에 시각화합니다.
plt.figure(figsize=(8, 6))
plt.scatter(X_circles[:, 0], X_circles[:, 1], c=y_kmeans_success, s=50, cmap='viridis')
plt.title('K-Means Succeeds After Feature Transformation')
# 가로, 세로 비율을 동일하게 설정하여 원이 제대로 보이게 합니다.
plt.gca().set_aspect('equal', adjustable='box')
plt.grid()
plt.show()

## 4. 최소제곱 데이터 피팅
최소제곱법은 모델의 예측값과 실제 데이터 값 사이의 오차(잔차)의 제곱합을 최소화하는 모델 파라미터 $\theta$를 찾는 방법입니다.
데이터에 가장 잘 맞는 모델을 찾기 위해 사용되며, 특히 선형 회귀 분석의 핵심 원리입니다.

수학적으로, 우리는 오차 벡터 $r = y - A\theta$의 L2-norm의 제곱, 즉 $\|y - A\theta\|^2$를 최소화하는 $\theta$를 찾고자 합니다.
$A$의 열들이 선형 독립일 때, 이 문제의 해는 **정규방정식(Normal Equations)**을 통해 유일하게 결정됩니다:

$$A^T A \hat{\theta} = A^T y$$

따라서, 최적의 파라미터 $\hat{\theta}$은 다음과 같이 구할 수 있습니다.

$$\hat{\theta} = (A^T A)^{-1} A^T y$$

### 4.1. 직선 피팅 (Straight-Line Fit)
가장 기본적인 예제로, 2차원 데이터를 가장 잘 설명하는 직선 모델 $y \\approx \\theta_0 + \\theta_1 x$의 파라미터 $\\theta = [\\theta_0, \\theta_1]$를 찾아보겠습니다.

이를 $y \\approx A\\theta$ 형태로 변환하기 위해, 각 데이터 포인트 $(x_i, y_i)$에 대해 설계 행렬(Design Matrix) $A$를 구성합니다.
직선 모델의 기저 함수(basis function)는 $f_0(x)=1$ (절편항)과 $f_1(x)=x$ (기울기항)이므로, $A$는 다음과 같은 $N \\times 2$ 행렬이 됩니다.

```
      [ 1  x₁ ]
  A = [ 1  x₂ ]
      [ :  :  ]
      [ 1  xₙ ]
```

In [ ]:
# 1. 직선 피팅을 위한 데이터 생성
np.random.seed(0) # 재현성을 위한 시드 고정
true_theta0 = 2.0  # 실제 y-절편
true_theta1 = 3.0  # 실제 기울기

num_data_points = 50
x_data = np.linspace(-1, 1, num_data_points)
noise = np.random.normal(0, 0.5, size=x_data.shape)
y_data = true_theta0 + true_theta1 * x_data + noise

# 2. 최소제곱 문제를 위한 설계 행렬 A와 벡터 y 구성
#    np.c_를 사용하여 열을 합칩니다.
A_fit = np.c_[np.ones(num_data_points), x_data]
print("설계 행렬 A (첫 5개 행):\n", A_fit[:5])
print("A 행렬의 크기:", A_fit.shape)

# 3. 정규방정식을 사용하여 최적의 파라미터 theta_hat 계산
#    theta_hat = inv(A.T @ A) @ A.T @ y
A_T_A = A_fit.T @ A_fit
A_T_y = A_fit.T @ y_data
theta_hat = np.linalg.inv(A_T_A) @ A_T_y

print("\n--- 찾은 파라미터 vs 실제 파라미터 ---")
print(f"           | {'찾은 값':^7s} | {'실제 값':^7s}")
print("------------------------------------")
print(f"절편 ($\theta_0$)  | {theta_hat[0]:^7.2f} | {true_theta0:^7.2f}")
print(f"기울기 ($\theta_1$) | {theta_hat[1]:^7.2f} | {true_theta1:^7.2f}")

# 4. 결과 시각화
plt.figure(figsize=(10, 6))
plt.scatter(x_data, y_data, label='Data Points', alpha=0.7)
plt.plot(x_data, theta_hat[0] + theta_hat[1] * x_data, color='red', linewidth=2, label='Fitted Line (Least Squares)')
plt.title('Straight-line Fit using Least Squares')
plt.xlabel('x')
plt.ylabel('y')
plt.legend()
plt.grid(True)
plt.show() 